In [0]:
---

title: 'DATA1001 Project #1'

author: "Ando Koki"

date: "4/14/2018"

output:

  html_document:

    code_folding: show

    highlight: tango

    number_section: yes

    theme: cosmo

    toc: yes

    toc_depth: 3

  pdf_document:

    toc: yes

    toc_depth: '3'

---



```{r setup, include=FALSE}

knitr::opts_chunk$set(echo = TRUE)

```

# Introduction

The pursuit of happiness has been a part of humanity longer than some may think; some even argue that it’s the reason we continue to do more than just exist. Evidence of this stems back to Ancient Greece, when Philosophers such as Aristotle wrote about it in many of his texts. In fact, the “pursuit of happiness” was a vital part of the United States Declaration of Independence, written in 1776. With Thomas Jefferson stating how he believed that happiness is attainable by gaining knowledge and living a self-sufficient

life surrounded by friends. If we look at the multitude of global religions, we see a similar importance placed on the idea of happiness.

 


# Data Source

The information was gathered from a single direct primary source: the Global Happiness Report created by the United Nations Sustainable Development Solutions Network (UNSDSN.)



# Research Questions

1. What are the different factors that lead to happiness of a country?

2. What similarities and differences do the countries experiencing the highest/lowest WHS have?

3. Does being in a certain region [continent] have any correlation to the average score of countries?

4. What role does the economy play in determining the happiness of a country?

5. Which factor, on average, contributes most/least to happiness?



# Analysis of Sources

 

**UNSDSN**



SDSN (Sustainable development solutions network) is a UN group run by scholars from around the world. They largely receive funding from government sources, mostly from European countries in departments relating to foreign affairs (naturally). Several firms, including BT (British Telecom), eni, Digicel, Glaxo Smith Kline, Novartis, Verizon,  and Ericsson also fund projects by SDSN. While the possibility of a bias is always possible, these companies likely stand to profit more from accurate international information pertaining to people’s desires, so it seems very unlikely that an agenda is being pushed meaning the data is very likely accurate.



**Gallup**



American analytics/data firm. The SDSN would have hired them for the specific task of conducting happiness surveys around the world. Being as large and reputable as they are, it’s in their best interest to make sure the information is collected properly. The happiness score was received from Gallup’s data, which was done by performing randomised phone surveys (that were still representative of the overall demographics of the country above 15 years old) in countries where that was a proven method. In countries where it wasn’t or where phone lines weren’t readily available country-wide, face to face surveys were conducted. Phone surveys were 15-30 minutes, face to face ones 30-60 minutes usually. Self reports are always questionable, but there’s no real objective way to measure happiness so we have to take them at face value. The information is unclear on how many people were interviewed.



# Discussion of Stakeholders

The people surveyed were asked the following:

 “Please imagine a ladder, with steps numbered from 0 at the bottom to 10 at the top. The top of the ladder represents the best possible life for you and the bottom of the ladder represents the worst possible life for you. On which step of  the ladder would you say you personally feel  you stand at this time?”



The report is carried out by experts of different fields from psychology, health, economics and more,

and looks at six key conditions of each country:

1. economic prosperity, including decent work for all who want it;

2. the physical and mental health of the citizens;

3. freedom of individuals to make key life decisions;

4. strong and vibrant social support networks (social capital);

5. shared public values of generosity; and

6. social trust, including confidence in the honesty of business and government.





# Code

## Preparation

### Packages

```{r}

library(data.table)

library(tidyverse)

library(corrplot)

library(plotly)

library(wildcard)

library(shiny)

library(PerformanceAnalytics)

library(DT)

```



### Data Importing & Cleaning

```{r}

wh15 <- fread("../input/2015.csv", data.table = FALSE)

wh16 <- fread("../input/2016.csv", data.table = FALSE)

wh17 <- fread("../input/2017.csv", data.table = FALSE)

wh15$year <- 2015

wh16$year <- 2016

wh17$year <- 2017



names(wh17)[2] <- "Happiness Rank"

names(wh17)[3] <- "Happiness Score"

names(wh17)[6] <- "Economy (GDP per Capita)"

names(wh17)[8] <- "Health (Life Expectancy)"

names(wh17)[11] <- "Trust (Government Corruption)"

names(wh17)[12] <- "Dystopia Residual"



wh15_17 <- bind_rows(wh15,wh16,wh17)

names(wh15_17)[5] <- "SD_error"

wh15_17 <- wh15_17 %>% select(Country:year,-Region, -SD_error)



names(wh15_17) <- c("Country","Happiness_Rank","Happiness_Score","Economy_GDP",

                 "Family","Health","Freedom","Trust",

                 "Generosity","Dystopia_Residual","year")

```



### Data Components

```{r}

names(wh15_17)

head(wh15_17)

str(wh15_17)

summary(wh15_17)

```





## Countries that did not appear in 3 years in a row

```{r}

countries.didnt.appear.3years <- wh15_17 %>% group_by(Country) %>% mutate(count = sum(year))

countries.didnt.appear.3years %>% filter(count != 6048) %>% select(Country, Happiness_Rank, year) %>% arrange(Country)

```

Some countries underwent a name change such as Hong Kong S.A.R., China to Hong Kong in 2017 for political reasons. While other countries—as admitted by the UNSDSN—did not fulfill the survey or gave the survey to its citizens. In order to minimize issues, the committee used the 2014 data of these countries. This of course affects the accuracy of the data, which was stated under the source validity.



## Analysis

### Correlation

#### Correlation check No.1

```{r}

corrplot(cor(wh15_17 %>% 

               select(Happiness_Score:Dystopia_Residual)), 

         method="color",  

         sig.level = 0.01, insig = "blank",

         addCoef.col = "black", 

         tl.srt=45, 

         type="upper"

         )

```

This correlation plot shows that the Economic GDP score tends to have the biggest impact to happiness score and the Health score has the second biggest impact.



#### Histogram of the World Happiness Scores from years 2015 to 2017

```{r}

hist(wh15_17$Happiness_Score , xlab = "World Happiness Score from 2015 to 2017", main = "World Happiness Score from 2015 to 2017")

```





### Ranking

```{r}

p <- ggplot(wh15_17 %>% filter(year==2017), aes(x= Happiness_Score,y= 

                                             reorder(Country,Happiness_Score))) + 

  geom_point(colour = "red", alpha = .5) + 

  geom_segment(aes(yend=reorder(Country, Happiness_Score)), xend = 0, colour="pink", alpha = .5) + 

  theme(axis.text.y = element_text(angle = 0, hjust = 1)) + 

  labs(title = "World Happiness Rank in 2017", y = "Country Name", x = "Happiness Score")

ggplotly(p)

```







#### 2017 Happiness Score World Map

```{r}

world <- map_data('world')

world <- world %>% filter(region != "Antarctica")

world <- fortify(world)

happiness.score17 <- wh15_17 %>% select(Country, Happiness_Score, year) %>% filter(year == 2017)

happiness.score17 <- wildcard(df = happiness.score17, wildcard = "United States", values = "USA",

         expand = TRUE, rules = NULL)

happiness.score17 <- wildcard(df = happiness.score17, wildcard = "United Kingdom", values = "UK",

         expand = TRUE, rules = NULL)



happiness.score17 <- wildcard(df = happiness.score17, wildcard = "Democratic Republic of the Congo", values = "Congo (Kinshasa)",

         expand = TRUE, rules = NULL)

ggplot() + 

  geom_map(data=world, map=world,

                  aes(x=long, y=lat, group=group, map_id=region),

                  fill="white", colour="black") + 

  geom_map(data=happiness.score17, map=world,

                  aes(fill=Happiness_Score, map_id=Country),

                  colour="black") + 

  scale_fill_continuous(low="red", high="yellow",

                               guide="colorbar") + 

  labs(title = "World Happiness Score in 2017")

```





World map - showing World at a glance and the state of global happiness



### Scatter Plot

#### Scatter Plot No.1

```{r}

plot_ly(data = wh15_17, 

        x=~Economy_GDP, y=~Happiness_Score, color=~Health, type = "scatter",

        text = ~paste("Country:", Country)) %>% 

        layout(title = "Happiness, GDP and Health relationship", 

               xaxis = list(title = "GDP per Capita"),

               yaxis = list(title = "Happiness Score"))

```



This interactive scatterplot shows that there is a strong positive correlation between GDP and Happiness Also points are coloured by the Health score, which also suggeests that Health tends to have big impact to happiness.



### Boxplot

#### Happiness score by region(boxplot)

```{r}

names(wh16)[4] <- "Happiness_Score"

ggplot(wh16, aes(x=Region, y= Happiness_Score, colour = Region)) + 

  geom_boxplot() + 

  theme(axis.text.x = element_text(angle = 60, hjust = 1)) + 

  labs(title = "Happiness Score Boxplot",

       x = "Region",

       y = "Happiness Score")

```





### Extract & Compare Top5, Middle5 and Worst5 Countries

```{r}

world.happiness17 <- wh15_17 %>% filter(year == 2017)

top5 <- world.happiness17 %>% head(5) %>% mutate(Level = "TOP5")

middle5 <- world.happiness17[76:80, ] %>% mutate(Level = "MIDDLE5")

worst5 <- world.happiness17 %>% tail(5) %>% mutate(Level = "WORST5")



comparison <- bind_rows(top5, middle5, worst5)



comparison$Level <- as.factor(comparison$Level)

comparison <- transform(comparison, Level = factor(Level, levels = c("TOP5", "MIDDLE5", "WORST5" )))

```



```{r}

datatable(comparison,

          options = list(

            lengthMenu = c(5, 10, 15)

          ),

          caption = 

            htmltools::tags$caption(

              style = 'caption-side: bottom; text-align: center;', 

              htmltools::em('Data table that only includes top5, middle5 and worst5 countries'))

          )

```



```{R}

comparison.score <- comparison %>% gather(key = "columns", value = "score", Happiness_Score:Dystopia_Residual)

comparison.score %>% 

  ggplot(aes(x = Level, y = score, colour = Level, fill = Level)) + 

  geom_boxplot(position=position_dodge(width=1)) + facet_wrap(~columns, scales = "free")

```





Analysis of the six different key conditions used to determine overall state of happiness of  a country. Looking at the top five, middle five and bottom five countries of the dataset.



# Conclusion

 

After analysing data of Global Happiness Levels in the world, created by the United Nations Sustainable Development Solutions Network, we were able to discover the impact of each different factor in determining “happiness.” We had also found that among the different factors, Economic GDP tends to have the greatest  on happiness with Health following close by. We then decided to focus on these two by establishing graphs of their relationships to discover that there is a direct relation between these factors. 



In addition to these facts, we decided to explore the topic deeper by classifying certain countries in the data to the top, median and bottom five countries according to ranked scores, in order to get a better sense of similarities and differences. The group determined that the “happiest” countries were located in Europe, particularly Scandinavia and Switzerland. Meanwhile the “least happy” countries were located in Africa and the Middle East. This suggests that countries in close proximity or those in the same region often have similar living conditions and are thus affected by factors similarly.



One bigger concern is how Trust has the lowest scores of all conditions looked at. Countries that have little to no trust and confidence in the governments, make it so that the citizens feel disenfranchised and are not able to take the life choices they wish, which is illustrated in the correlation between low trust and low Freedom scores.



By looking at and analysing these reports, we are able to decipher what makes countries and their citizens happier, thus allowing us to focus on prioritizing and improving these aspects of each nation. It is through this that we are able to achieve the true pursuit of happiness, which we as human beings strive for.




